In [1]:
import pandas as pd
import re
from os import system, listdir
from random import shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from random import shuffle
from joblib import dump, load

directory = './aclImdb/train/neg'
neg = []

for filename in os.listdir(directory):
        f = open(f'{directory}/{filename}', encoding='utf-8')
        lines = f.read()
        neg.append(lines)

directory = './aclImdb/test/pos'
pos = []

for filename in os.listdir(directory):
        f = open(f'{directory}/{filename}', encoding='utf-8')
        lines = f.read()
        pos.append(lines)

directory = './aclImdb/test/neg'


for filename in os.listdir(directory):
        f = open(f'{directory}/{filename}', encoding='utf-8')
        lines = f.read()
        neg.append(lines)

directory = './aclImdb/train/pos'
for filename in os.listdir(directory):
        f = open(f'{directory}/{filename}', encoding='utf-8')
        lines = f.read()
        pos.append(lines)


negative = pd.DataFrame(neg)
negative["label"] = 0
positive = pd.DataFrame(pos)
positive["label"] = 1
all = pd.concat([negative, positive])
all.fillna(0, inplace=True)
all = all.rename(columns={0: "text"}).reset_index(drop=True)
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
all['text'] = all['text'].apply(remove_punctuations)
all = all.sample(frac=1).reset_index(drop=True)
all

,text,label
0,This movie has some things that are pretty ama...,1
1,Duchess and her three kittens are enjoying the...,1
2,The Class is a comedy series that portrays a b...,0
3,Latter days is the best gay movie of the homos...,1
4,There is part of one sequence where some water...,0
...,...,...
49995,This movie succeeds at being one of the most u...,0
49996,There is a reason Chairman of the Board got a ...,0
49997,My Favorite part was when the credits started ...,0
49998,Jack Frost no kids its not the warm hearted fa...,0


In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.sparse import save_npz, load_npz

In [11]:
vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words='english')
vectorizer.fit(all['text'].values)
dump(vectorizer, './Preprocessing/vectorizer.joblib')
vectorized = vectorizer.transform(all['text'].values)
save_npz('vectorized.npz', vectorized)
idf_transformer = TfidfTransformer()
idf_transformer.fit(vectorized)
dump(idf_transformer, './Preprocessing/idf_transformer.joblib')
transformed = idf_transformer.transform(vectorized)
save_npz('transformed.npz', transformed)
listofwords = vectorizer.get_feature_names()

In [18]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import numpy as np

y = all['label'].values

X_train, X_test, y_train, y_test = train_test_split(transformed, y)

model = SGDClassifier()
model.fit(X_train, y_train)
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
prediction = model.predict(X_test)
print(train_score, test_score)
comparison = zip(prediction,y_test)
comp_df = pd.DataFrame(comparison)
comp_df.head(20)
dump(model, './Model/model.joblib')

0.9318666666666666 0.89384


['model.joblib']